# Capstone Project

The purpose of this project is to create a heatmap of the Milwaukee Police Department's Use of Force incidents performed during the first three quarters of 2020 (the most recent data of this type available). Data is from the City of Milwaukee's Open Data Portal.

In [360]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library
from folium.plugins import HeatMap

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Libraries imported.


## Identify Datasets in the Milwaukee Data Portal

The Milwaukee Police Department (MPD) splits their information up by quarter, so each subject has a separate 2020Q1, 2020Q2, and 2020Q3 components. To use them, these components will need to be combined to show all data currently provided for 2020 (no Q4 data has been published yet). 

Also, each quarter's data is split into several different tables which provide little context on their own. In order to get the location of each use of force, tables must be joined to each other (through an additional table) in order to get key columns to match up. Here are the following tables required for each quarter, listing their useful columns:

**AIM_Use_of_Force**  each record represents a use of force by MPD, though some correspond to the same case.  
  *CAD_Call_Number* : matches the *DocumentPoliceNumber* in **Tracs_ContactSummary_Joined**

**Tracs_ContactSummary_Joined**  details for traffic related police activity
  *DocumentPoliceNumber* :  matches *CAD_Call_Number* in **AIM_Use_of_Force**  
  *Location_ColKey* : matches *CollKey* in **Tracs_Location**
  
**Tracs_NTC_Joined**  details for non-traffic related police activity
  *DocumentPoliceNumber* :  matches *CAD_Call_Number* in **AIM_Use_of_Force**  
  *Location_ColKey* : matches *CollKey* in **Tracs_Location**
  
**Tracs_Warning_Joined**  details for warning related police activity
  *DocumentPoliceNumber* :  matches *CAD_Call_Number* in **AIM_Use_of_Force**  
  *Location_ColKey* : matches *CollKey* in **Tracs_Location**
  
**Tracs_ELCI_Joined**  details for electric citation related police activity
  *DocumentPoliceNumber* :  matches *CAD_Call_Number* in **AIM_Use_of_Force**  
  *Location_ColKey* : matches *CollKey* in **Tracs_Location**

**Tracs_Location**  
  *CollKey* : matches *Location_CollKey* in **Tracs_ContactSummary_Joined**  
  *Latitude*  
  *Longitude*
  
In total, fifteen tables need to be combined to unite each use of force record with its latitude and longitude, since each of the three quarters' data is split into three tables, and there are five tables per quarter.


## Gather data - Quarter 1 2020

In [361]:
q1force = pd.read_csv('https://data.milwaukee.gov/dataset/9fa186c6-fb69-4cb7-8865-f8fcba1ae2fd/resource/04e4d764-28aa-4c17-a29a-c05532c3901f/download/copy-of-aim-use-of-force-q1-2020-redacted.csv')
q1force.head()

,UOF Report Number,incident date,CAD CLEAN,Officer Activity,Type of Force,IncidentPersonID
0,PF-2020-0010,1/4/2020,200040248.0,Effecting ArrestTraffic Stop,Bodily Force,Person 1
1,PF-2020-0018,1/13/2020,200132357.0,Effecting ArrestField Interview,Bodily Force,Person 1
2,PF-2020-0018,1/13/2020,200132357.0,Effecting ArrestField Interview,Bodily Force,Person 1
3,PF-2020-0018,1/13/2020,200132357.0,Effecting ArrestField Interview,Bodily Force,Person 1
4,PF-2020-0018,1/13/2020,200132357.0,Effecting ArrestField Interview,Bodily Force,Person 1


In [362]:
# Number of records
q1force.shape[0]

72

In [363]:
# rename some columns to match the headers of Q2 and Q3 tables
q1force.rename(columns={'UOF Report Number': 'UOF_Report_Number', 'incident date': 'incident_date','CAD CLEAN':'CAD_Call_Number', 'Officer Activity':'Officer_Activity', 'Type of Force':'Type_of_Force' }, inplace=True)
q1force.head()

,UOF_Report_Number,incident_date,CAD_Call_Number,Officer_Activity,Type_of_Force,IncidentPersonID
0,PF-2020-0010,1/4/2020,200040248.0,Effecting ArrestTraffic Stop,Bodily Force,Person 1
1,PF-2020-0018,1/13/2020,200132357.0,Effecting ArrestField Interview,Bodily Force,Person 1
2,PF-2020-0018,1/13/2020,200132357.0,Effecting ArrestField Interview,Bodily Force,Person 1
3,PF-2020-0018,1/13/2020,200132357.0,Effecting ArrestField Interview,Bodily Force,Person 1
4,PF-2020-0018,1/13/2020,200132357.0,Effecting ArrestField Interview,Bodily Force,Person 1


In [364]:
q1summary = pd.read_csv('https://data.milwaukee.gov/dataset/9fa186c6-fb69-4cb7-8865-f8fcba1ae2fd/resource/c178f5b5-d4e5-4c23-95b0-d829f8d44ada/download/copy-of-tracs-contact-summary-joined-formatted.csv')
q1summary.head()

,PrdKey,summaryDateOccurred,summaryTimeOccurred,summaryReason,summaryReasonDetail,summaryOtherReason,summaryOtherReasonDetail,summaryOutcome,agencyAgencySpace,documentPoliceNumber,LocationColKey
0,Formcontactsum00737100601280202008461462274311114,1/28/2020,8:43,TS01,VR01,NaN,NaN,"02,03","DEF DRINVING LISTED AUTO, S/B ON N. 27TH ST @ ...",200280642,Collection00737100601280202008461534347311115#...
1,Formcontactsum00737100602060202001343369944311473,2/6/2020,13:06,TS01,"OR01,SV01",NaN,NaN,4,"I OBSERVED DEF SPEEDING 51MPH ,30 SPEED ZONE A...",200371284,Collection00737100602060202001253863515311237#...
2,Formcontactsum00737100602100202001512366734311905,2/10/2020,13:42,TS01,SV01,NaN,NaN,4,"DEF. DRIVING LISTED AUTO,STOPPED FOR SPEEDING....",200411328,Collection00737100602100202001512443096311906#...
3,Formcontactsum00818800601300202008440868943278730,1/30/2020,8:37,TS01,SV01,NaN,NaN,3,LISTED SUBJECT WAS OBSERVED TRAVELING ABOVE TH...,200300628,Collection00818800601300202008400444419278489#...
4,Formcontactsum0083920060206020201104446856377978,2/6/2020,7:20,FI01,99,NaN,SUSPECT INVOLVED IN A DISORDERLY ACT,4,SUBJECT WENT INTO HER CHILD SCHOOL LOOKING FOR...,200370549,Collection0083920060206020201108162775477982#U...


In [365]:
# Number of records
q1summary.shape[0]

15330

In [366]:
q1nontraffic = pd.read_csv('https://data.milwaukee.gov/dataset/9fa186c6-fb69-4cb7-8865-f8fcba1ae2fd/resource/c08f0515-ea70-407a-954d-e9fedd0cc119/download/copy-of-tracs-ntc-joined-formatted.csv')
q1nontraffic.head()

,PrdKey,violationDateOccurred,violationTimeOccurred,violationLocalOrdinanceDescription,violationLocalOrdinanceNumber,violationStatuteDescription,violationStatuteNumber,agencyAgencySpace,LocationColKey,DefendantColKey,documentPoliceNumber\t\t\t\t\t\t\t
0,Formntc.frm0128440060118020200445439818107345,1/18/2020,16:32,ADULT-RETAIL THEFT- VALUE LESS THAN 99.99,110-35-1,NaN,NaN,DEFENDANT TOOK ITEMS THAT WERE INTENDED FOR RE...,Collection012844006011802020045600798391073117...,Collection01284400601180202004460116289107379#...,2-0181246\t\t\t\t\t\t\t
1,Formntc.frm01409400602150202003183176134108345,2/15/2020,14:30,ADULT-VANDALISM,110-15,NaN,NaN,SUBJECT INTENTIONALLY SMASHED A SMALL LIVING R...,Collection014094006021502020032251687091083129...,Collection01409400602150202003184725565108380#...,200461397\t\t\t\t\t\t\t
2,Formntc.frm01530200602110202002381040948106980,2/11/2020,14:00,ADULT- RETAIL THEFT VALUE BTWN $100 & $500,110-35-1,NaN,NaN,A KNOWN ACTOR INTENTIONALLY AND WITHOUT CONSEN...,Collection015302006021102020024323690491069152...,Collection015302006021102020023829145281069114...,200421471\t\t\t\t\t\t\t
3,Formntc.frm0171050060219020200246599818106064,2/17/2020,14:00,ADULT-VANDALISM,110-15,NaN,NaN,LISTED SUBJECT INTENTIONALLY DAMAGED THE VICTI...,Collection017105006021902020025211375461060493...,Collection017105006021902020024859659711060242...,200501415\t\t\t\t\t\t\t
4,Formntc.frm017912006012302020022825427231080127,1/23/2020,1:50,JUV - CURFEW LOIT OF MINOR UNDER 17,106-23,NaN,NaN,LISTED SUBJECT WAS UNDER 17 AND WALKING THROUG...,Collection017912006012302020023454367521080199...,Collection017912006012302020022941885471080161...,200230147\t\t\t\t\t\t\t


In [367]:
# rename column to match other quarters
q1nontraffic.rename(columns={'documentPoliceNumber\t\t\t\t\t\t\t': 'documentPoliceNumber'}, inplace=True)
q1nontraffic.head()

,PrdKey,violationDateOccurred,violationTimeOccurred,violationLocalOrdinanceDescription,violationLocalOrdinanceNumber,violationStatuteDescription,violationStatuteNumber,agencyAgencySpace,LocationColKey,DefendantColKey,documentPoliceNumber
0,Formntc.frm0128440060118020200445439818107345,1/18/2020,16:32,ADULT-RETAIL THEFT- VALUE LESS THAN 99.99,110-35-1,NaN,NaN,DEFENDANT TOOK ITEMS THAT WERE INTENDED FOR RE...,Collection012844006011802020045600798391073117...,Collection01284400601180202004460116289107379#...,2-0181246\t\t\t\t\t\t\t
1,Formntc.frm01409400602150202003183176134108345,2/15/2020,14:30,ADULT-VANDALISM,110-15,NaN,NaN,SUBJECT INTENTIONALLY SMASHED A SMALL LIVING R...,Collection014094006021502020032251687091083129...,Collection01409400602150202003184725565108380#...,200461397\t\t\t\t\t\t\t
2,Formntc.frm01530200602110202002381040948106980,2/11/2020,14:00,ADULT- RETAIL THEFT VALUE BTWN $100 & $500,110-35-1,NaN,NaN,A KNOWN ACTOR INTENTIONALLY AND WITHOUT CONSEN...,Collection015302006021102020024323690491069152...,Collection015302006021102020023829145281069114...,200421471\t\t\t\t\t\t\t
3,Formntc.frm0171050060219020200246599818106064,2/17/2020,14:00,ADULT-VANDALISM,110-15,NaN,NaN,LISTED SUBJECT INTENTIONALLY DAMAGED THE VICTI...,Collection017105006021902020025211375461060493...,Collection017105006021902020024859659711060242...,200501415\t\t\t\t\t\t\t
4,Formntc.frm017912006012302020022825427231080127,1/23/2020,1:50,JUV - CURFEW LOIT OF MINOR UNDER 17,106-23,NaN,NaN,LISTED SUBJECT WAS UNDER 17 AND WALKING THROUG...,Collection017912006012302020023454367521080199...,Collection017912006012302020022941885471080161...,200230147\t\t\t\t\t\t\t


In [368]:
# remove excess letters from the documentPoliceNumber column
q1nontraffic['documentPoliceNumber']= q1nontraffic['documentPoliceNumber'].str[:9]
q1nontraffic.head()

,PrdKey,violationDateOccurred,violationTimeOccurred,violationLocalOrdinanceDescription,violationLocalOrdinanceNumber,violationStatuteDescription,violationStatuteNumber,agencyAgencySpace,LocationColKey,DefendantColKey,documentPoliceNumber
0,Formntc.frm0128440060118020200445439818107345,1/18/2020,16:32,ADULT-RETAIL THEFT- VALUE LESS THAN 99.99,110-35-1,NaN,NaN,DEFENDANT TOOK ITEMS THAT WERE INTENDED FOR RE...,Collection012844006011802020045600798391073117...,Collection01284400601180202004460116289107379#...,2-0181246
1,Formntc.frm01409400602150202003183176134108345,2/15/2020,14:30,ADULT-VANDALISM,110-15,NaN,NaN,SUBJECT INTENTIONALLY SMASHED A SMALL LIVING R...,Collection014094006021502020032251687091083129...,Collection01409400602150202003184725565108380#...,200461397
2,Formntc.frm01530200602110202002381040948106980,2/11/2020,14:00,ADULT- RETAIL THEFT VALUE BTWN $100 & $500,110-35-1,NaN,NaN,A KNOWN ACTOR INTENTIONALLY AND WITHOUT CONSEN...,Collection015302006021102020024323690491069152...,Collection015302006021102020023829145281069114...,200421471
3,Formntc.frm0171050060219020200246599818106064,2/17/2020,14:00,ADULT-VANDALISM,110-15,NaN,NaN,LISTED SUBJECT INTENTIONALLY DAMAGED THE VICTI...,Collection017105006021902020025211375461060493...,Collection017105006021902020024859659711060242...,200501415
4,Formntc.frm017912006012302020022825427231080127,1/23/2020,1:50,JUV - CURFEW LOIT OF MINOR UNDER 17,106-23,NaN,NaN,LISTED SUBJECT WAS UNDER 17 AND WALKING THROUG...,Collection017912006012302020023454367521080199...,Collection017912006012302020022941885471080161...,200230147


In [369]:
# Number of records
q1nontraffic.shape[0]

2218

In [370]:
q1warning = pd.read_csv('https://data.milwaukee.gov/dataset/9fa186c6-fb69-4cb7-8865-f8fcba1ae2fd/resource/be57487c-7439-4810-a91f-49075e47f52f/download/copy-of-tracs-warning-joined-formatted.csv')
q1warning.head()

,PrdKey,documentPoliceNumber,summaryDateOccurred,summaryTimeOccurred,IndividualColKey,LocationColKey,agencyAgencySpace
0,Formwarning.fr01476700602250202002215641585106...,200561443,2/25/2020,14:21,Collection014767006022502020022222341251069257...,Collection014767006022502020022524703071069281...,SUBJECT WAS OBSERVED DRIVING A UNREGISTERED AU...
1,Formwarning.fr01476800601130202006094926325106...,200132058,1/13/2020,18:09,Collection014768006011302020061037532801060594...,Collection01476800601130202006173970629106057#...,NaN
2,Formwarning.fr0155100060130020200551169818105544,200302136,1/30/2020,17:51,Collection01551000601300202005514249903105547#...,Collection01551000601300202005554830045105584#...,[REDACTED] OBSRVD ABOVE VEH WITH EXPIRED REG 0...
3,Formwarning.fr01551000602060202009220372758105...,200372597,2/6/2020,21:22,Collection015510006020602020092228468751055170...,Collection015510006020602020092516218441055206...,[REDACTED] OBSERVED ABOVE VEH WITH EXPIRED REG...
4,Formwarning.fr0155100060211020200924169818105544,200422552,2/11/2020,21:24,Collection01551000602110202009245349903105547#...,Collection0155100060211020200929199091105583#U...,[REDACTED]OBSERVED ABOVE LISTED VEH WITH SUSP ...


In [371]:
q1warning.shape[0]

8379

In [372]:
q1location = pd.read_csv('https://data.milwaukee.gov/dataset/9fa186c6-fb69-4cb7-8865-f8fcba1ae2fd/resource/4cbfb8ad-f198-4434-a97e-87cc17583a43/download/copy-of-tracs-location-formatted.csv')
q1location.head()

,CollKey,latitude,longitude
0,Collection0051660060123020200844046856305581#U...,43.052939,-87.893371
1,Collection0051660060123020200858589603055352#U...,43.052648,-87.892626
2,Collection0051660060221020200325475758805557#U...,43.052940,-87.893257
3,Collection007369006021402020042548469051142293...,42.988300,-87.905472
4,Collection00737100601050202011145011392311397#...,43.026032,-87.992432


In [373]:
q1electronic = pd.read_csv('https://data.milwaukee.gov/dataset/9fa186c6-fb69-4cb7-8865-f8fcba1ae2fd/resource/5dbd7316-87f8-4b01-9081-b4aa909fa459/download/copy-of-tracs-elci-joined-formatted.csv')
q1electronic.head()

,PrdKey,violationDateOccurred,violationTimeOccurred,violationLocalOrdinanceDescription,violationLocalOrdinanceNumber,violationStatuteDescription,violationStatuteNumber,agencyAgencySpace,LocationColKey,DefendantColKey,documentPoliceNumber
0,Formelci.frm01074700601020202011530447593485269,1/2/2020,10:23,822 OAS,101-1-2,OPERATING AFTER SUSPENSION,343.44(1)(a),NaN,Collection01074700601020202011540334746485400#...,Collection01074700601020202011540437184485446#...,200020852
1,Formelci.frm0107470060107020200228248446485423,1/7/2020,13:45,822 OAS,101-1-2,OPERATING AFTER SUSPENSION,343.44(1)(a),NaN,Collection01074700601070202002282662459485448#...,Collection01074700601070202002175646607485325#...,200071375
2,Formelci.frm0107470060129020200305444622148545,1/29/2020,13:59,101 OAI COMBO 1ST,101-1-2,OPERATING WHILE UNDER THE INFLUENCE,346.63(1)(a),NaN,Collection01074700601290202004435214528485111#...,Collection0107470060129020200307263004548580#U...,200291509
3,Formelci.frm014202006012102020010705893941055400,1/21/2020,12:51,822 OAS,101-1-2,OPERATING AFTER SUSPENSION,343.44(1)(a),NaN,Collection014202006012102020010707108611055425...,Collection014202006012102020010322231611055381...,200211218
4,Formelci.frm014202006012102020023248422121055697,1/21/2020,14:32,484 EX POST LIM 25-29 WRK ZONE $272.80,101-1-2,"EXCEEDING SPEED ZONES, ETC. (25-29 MPH)",346.57(5),NaN,Collection014202006012102020025537706881055753...,Collection014202006012102020023555603941055728...,200211477


In [374]:
q1electronic.shape[0]

13924

In [375]:
# Number of records
q1location.shape[0]

39759

## Gather data - Quarter 2 2020

In [376]:
q2force = pd.read_csv('https://data.milwaukee.gov/dataset/b0e4a98c-ede6-4136-a78a-9d116ada1eef/resource/5b482cdf-790c-47ce-9f8d-c885d62127ff/download/aim_use_of_force-redacted.csv')
q2force.head()

,UOF_Report_Number,incident_date,CAD_Call_Number,Officer_Activity,Type_of_Force,Justification_for_UOF,IncidentPersonID
0,PF-2020-0197,4/23/2020,201141894.0,Dispatched AssignmentEffecting ArrestField Int...,ECDBodily Force,NaN,Person 1
1,PF-2020-0197,4/23/2020,201141894.0,Dispatched AssignmentField Interview,Bodily Force,NaN,Person 1
2,PF-2020-0172,4/15/2020,201061190.0,Traffic Stop,ECD,NaN,Person 1
3,PF-2020-0244,5/26/2020,201472687.0,Effecting ArrestFoot PursuitInvestigationTraff...,Bodily Force,Make lawful arrestPrevent Escape,Person 1
4,PF-2020-0244,5/26/2020,201472687.0,Effecting ArrestFoot PursuitInvestigationTraff...,Bodily Force,Make lawful arrestPrevent Escape,Person 1


In [377]:
q2force.shape[0]

58

In [378]:
q2summary = pd.read_csv('https://data.milwaukee.gov/dataset/b0e4a98c-ede6-4136-a78a-9d116ada1eef/resource/da63f81a-a4fd-432c-86bd-417763192620/download/tracs_contactsummary_joined-redacted.csv')
q2summary.head()

,PrdKey,summaryDateOccurred,summaryTimeOccurred,summaryReason,summaryReasonDetail,summaryOtherReason,summaryOtherReasonDetail,summaryOutcome,agencyAgencySpace,documentPoliceNumber,LocationColKey
0,Formcontactsum00737100605010202008235011392311237,5/1/2020,8:13,TS01,SV01,NaN,NaN,4,"OBSERVED DEF DRIVING LISTED AUTO, SPEEDING, 66...",201220571,Collection00737100605010202008182520747311888#...
1,Formcontactsum01022100604110202002043648684250116,4/11/2020,13:45,TS01,SV01,NaN,NaN,4,OBSERV VEH W/B ON W. BOBOLINK AVE SPEEDING 48 ...,201021145,Collection0102210060411020200157387880925089#U...
2,Formcontactsum01022100604160202001014253336250224,4/16/2020,12:31,TS01,OR01,NaN,NaN,4,OBSERV VEH S/B ON N. 103RD ST DISREG STOP SIGN...,201071112,Collection01022100604160202012544386492250206#...
3,Formcontactsum01022100604160202001583371206250432,4/16/2020,13:47,TS01,OR01,NaN,NaN,4,OBSERV VEH S/B ON N. 103RD ST DISREG STOP SIGN...,201071288,Collection01022100604160202001573457694250425#...
4,Formcontactsum01022100604200202001271373970250685,4/20/2020,13:16,TS01,SV01,NaN,NaN,4,OBSERV VEH W/B ON W. BOBOLINK AVE SPEEDING 46 ...,201111184,Collection01022100604200202001244827513250579#...


In [379]:
q2summary.shape[0]

6454

In [380]:
q2nontraffic = pd.read_csv('https://data.milwaukee.gov/dataset/b0e4a98c-ede6-4136-a78a-9d116ada1eef/resource/e683f478-2ca6-4ad7-87dc-bd903276a5b9/download/tracs_ntc_joined-redacted.csv')
q2nontraffic.head()

,PrdKey,violationDateOccurred,violationTimeOccurred,violationLocalOrdinanceDescription,violationLocalOrdinanceNumber,violationStatuteDescription,violationStatuteNumber,agencyAgencySpace,LocationColKey,DefendantColKey,documentPoliceNumber
0,FormntcV3.frm0284970060403020200631503743570047,4/3/2020,5:14,O900440,106-31-1,NaN,NaN,[REDACTED]. LISTED SUBJ OBSERVED LOITERING ON...,Collection0284970060403020200643122074770093#U...,Collection028497006040302020064120909170079#UN...,200940413.0
1,FormntcV3.frm03127300604040202005572415067113958,4/4/2020,17:57,O900543,105-138-2,NaN,NaN,DEFENDANT INTENTIONALLY AND WITHOUT CONSENT OB...,Collection031273006040402020090300385841139136...,Collection03127300604040202005590330045113984#...,200951524.0
2,FormntcV3.frm0291970060405020200245057156063557,4/5/2020,1:34,O900385,106-38-2,NaN,NaN,LISTED SUBJECT HAD A CLEAR CORNER CUT BAGGIE W...,Collection0291970060405020200248243079263587#U...,Collection0291970060405020200246537880963585#U...,200960166.0
3,FormntcV3.frm0263400060405020200849448911892753,4/5/2020,18:19,O900385,106-38-2,NaN,NaN,SUBJECT WAS IN POSSESSION OF A GREEN PLANT MAT...,Collection02634000604050202008545484067927100#...,Collection0263400060405020200851583004592784#U...,200961696.0
4,FormntcV3.frm0288000060406020200347388911893153,4/6/2020,14:53,O901058,110-35-1,NaN,NaN,ABOVE LISTED DEFENDANT INTENTIONALLY AND WITHO...,Collection0288000060406020200438237072793198#U...,Collection0288000060406020200430106856393181#U...,200971412.0


In [381]:
q2nontraffic.shape[0]

324

In [382]:
q2warning = pd.read_csv('https://data.milwaukee.gov/dataset/b0e4a98c-ede6-4136-a78a-9d116ada1eef/resource/33a4ec7a-ad7b-4f29-b2b1-4ef260474642/download/tracs_warning_joined-redacted.csv')
q2warning.head()

,PrdKey,documentPoliceNumber,summaryDateOccurred,summaryTimeOccurred,IndividualColKey,LocationColKey,agencyAgencySpace
0,Formwarning.fr01510300605060202012401537188210398,201271116,5/6/2020,12:40,Collection01510300605060202012404588593210690#...,Collection0151030060506020201243211333221095#U...,NaN
1,Formwarning.fr01510300605090202004584351196210189,201301652,5/9/2020,16:58,Collection0151030060509020200459046737210409#U...,Collection01510300605090202005004740110210329#...,NaN
2,Formwarning.fr01510300605090202012310790694210887,201301011,5/9/2020,12:31,Collection01510300605090202012313622268210179#...,Collection0151030060509020201232463213210823#U...,NaN
3,Formwarning.fr01510300605230202003564745113210324,201441429,5/23/2020,15:56,Collection01510300605230202003572033044210642#...,Collection01510300605230202003593554456210832#...,NaN
4,Formwarning.fr01510300605260202003355212587210128,201471708,5/26/2020,15:35,Collection01510300605260202003361080778210320#...,Collection01510300605260202003371147649210880#...,NaN


In [383]:
q2warning.shape[0]

3621

In [384]:
q2electronic = pd.read_csv('https://data.milwaukee.gov/dataset/b0e4a98c-ede6-4136-a78a-9d116ada1eef/resource/89b1a86c-239f-4860-8211-94f269618f99/download/tracs_elci_joined-redacted.csv')
q2electronic.head()

,PrdKey,violationDateOccurred,violationTimeOccurred,violationLocalOrdinanceDescription,violationLocalOrdinanceNumber,violationStatuteDescription,violationStatuteNumber,agencyAgencySpace,LocationColKey,DefendantColKey,documentPoliceNumber
0,Formelci.frm00533900607220201710170873701321119,5/4/2020,10:33,NaN,NaN,NaN,NaN,NaN,NaN,Collection00533900608100201701271184976265242#...,172030651
1,Formelci.frm01510300604300202012555261036210707,4/30/2020,12:55,483 EX POST LIM 20-24 WRK ZONE $222.40,101-1-2,"EXCEEDING SPEED ZONES, ETC. (20-24 MPH)",346.57(5),NaN,Collection01510300604300202012575061091210806#...,Collection01510300604300202012561010450210901#...,201211157
2,Formelci.frm01510300605060202001100844535210410,5/6/2020,13:10,482 EX POST LIM 16-19 WRK ZONE $172.00,101-1-2,"EXCEEDING SPEED ZONES, ETC. (16-19 MPH)",346.57(5),NaN,Collection01510300605060202001134711659210412#...,Collection0151030060506020200110326143210658#U...,201271222
3,Formelci.frm01510300605060202003252620348210459,5/6/2020,15:25,483 EX POST LIM 20-24 WRK ZONE $222.40,101-1-2,"EXCEEDING SPEED ZONES, ETC. (20-24 MPH)",346.57(5),NaN,Collection01510300605060202003274220815210711#...,Collection01510300605060202003254465058210653#...,201271576
4,Formelci.frm01510300605070202004422053100210369,5/7/2020,16:42,482 EX POST LIM 16-19 WRK ZONE $172.00,101-1-2,"EXCEEDING SPEED ZONES, ETC. (16-19 MPH)",346.57(5),NaN,Collection01510300605070202004442220280210528#...,Collection01510300605070202004423424653210526#...,201281675


In [385]:
q2electronic.shape[0]

7352

In [386]:
q2location = pd.read_csv('https://data.milwaukee.gov/dataset/b0e4a98c-ede6-4136-a78a-9d116ada1eef/resource/1aa96c56-4f3e-4218-b6f6-8a70a33b7d3b/download/tracs_location.csv')
q2location.head()

,CollKey,latitude,longitude
0,Collection00737100604250202011235349886311596#...,43.063307,-87.940048
1,Collection00737100604250202011270432305311821#...,43.063307,-87.940048
2,Collection00737100604250202011282136899311995#...,43.063307,-87.940048
3,Collection00737100604250202011292450862311157#...,43.063307,-87.940048
4,Collection00737100604260202011223921137311864#...,43.051423,-87.947781


In [387]:
q2location.shape[0]

17522

## Gather data, Quarter 3 2020

In [388]:
q3force = pd.read_csv('https://data.milwaukee.gov/dataset/930c7e1a-793f-451c-b1dc-ee55946a0545/resource/c4edfeea-a15f-4312-9f60-2fba8aafaeae/download/aim-use-of-force.csv')
q3force.head()

,UOF_Report_Number,incident_date,CAD_Call_Number,Officer_Activity,Type_of_Force,Justification_for_UOF,IncidentPersonID
0,PF-2020-0315,7/2/2020,201842801.0,Traffic Stop,Bodily Force,Achieve or maintain control of resistive subject,Person 1
1,PF-2020-0315,7/2/2020,201842801.0,Traffic Stop,Bodily Force,Achieve or maintain control of resistive subject,Person 1
2,PF-2020-0318,7/5/2020,201872089.0,Dispatched AssignmentEffecting ArrestField Int...,Bodily Force,Achieve or maintain control of resistive subje...,Person 1
3,PF-2020-0318,7/5/2020,201872089.0,Dispatched AssignmentEffecting ArrestField Int...,Bodily ForceHandcuffing,Achieve or maintain control of resistive subje...,Person 1
4,PF-2020-0320,7/6/2020,201880508.0,Dispatched AssignmentEffecting ArrestField Int...,Bodily Force,Achieve or maintain control of resistive subje...,Person 1


In [389]:
q3force.shape[0]

44

In [390]:
q3summary = pd.read_csv('https://data.milwaukee.gov/dataset/930c7e1a-793f-451c-b1dc-ee55946a0545/resource/378709c9-dbfa-494a-ba35-b3df0ee54a2f/download/tracs-contactsummary-joined.csv')
q3summary.head()

,PrdKey,summaryDateOccurred,summaryTimeOccurred,summaryReason,summaryReasonDetail,summaryOtherReason,summaryOtherReasonDetail,summaryOutcome,agencyAgencySpace,documentPoliceNumber,LocationColKey
0,Formcontactsum00937000607250202010530316696107...,7/25/2020,10:42,TS01,SV01,NaN,NaN,4,MP D5 8-4PM SEE CASE INFORMATION,202070823,Collection009370006072502020104619205061072239...
1,Formcontactsum00937000609040202002265741103107...,9/4/2020,14:21,TS01,OR01,NaN,NaN,3,MP D5 8-4PM FAILURE TO YEILD RIGHT OF WAY ON ...,202481212,Collection009370006090402020022526210061072422...
2,Formcontactsum01022100608190202003081948392120...,8/19/2020,14:52,TS01,OR01,NaN,NaN,4,SEE CASE INFORMATION,202321300,Collection01022100608190202003055537904120693#...
3,Formcontactsum01022100608190202003410145403120...,8/19/2020,15:29,TS01,OR01,NaN,NaN,4,SEE CASE INFORMATION,202321370,Collection010221006081902020033625392441206318...
4,Formcontactsum01022100608190202011560582232149...,8/19/2020,10:50,TS01,"OR01,SV01",NaN,NaN,"04,05",SEE CASE INFORMATION,202320777,Collection01022100608190202011451631655149987#...


In [391]:
q3summary.shape[0]

7594

In [392]:
q3nontraffic = pd.read_csv('https://data.milwaukee.gov/dataset/930c7e1a-793f-451c-b1dc-ee55946a0545/resource/ba592ff8-8714-40c3-9d18-5bc7ca66b5ae/download/tracs-ntc-joined.csv')
q3nontraffic.head()

,PrdKey,violationDateOccurred,violationTimeOccurred,violationLocalOrdinanceDescription,violationLocalOrdinanceNumber,violationStatuteDescription,violationStatuteNumber,agencyAgencySpace,LocationColKey,DefendantColKey,documentPoliceNumber
0,FormntcV3.frm01842900607040202003273289118163653,7/4/2020,1:31,O900543,105-138-2,NaN,NaN,"[REDACTED], LOC-63, SUBJECT FLED FROM UNIFORM ...",Collection01842900607040202003542225565163680#...,Collection01842900607040202003480243702163656#...,201860181.0
1,FormntcV3.frm01760200607040202012122515067104756,7/4/2020,12:12,O900282,118-80-1,NaN,NaN,I OBSERVED THE LISTED VESSEL TRAVELING AT A SP...,Collection01760200607040202012183332086104797#...,Collection01760200607040202012153744243104784#...,201861085.0
2,FormntcV3.frm03127800607050202004424819933100464,7/5/2020,3:31,O900385,106-38-2,NaN,NaN,SUBJECT WAS OBSERVED IN POSESSION OF A GREEN L...,Collection03127800607050202004113868833100462#...,Collection031278006070502020060819273581225113...,201870549.0
3,FormntcV3.frm028795006070602020075115390631500110,7/6/2020,7:51,O900454,106-23,NaN,NaN,[REDACTED] LISTED SUBJECT IS UNDER 17 OUTSIDE ...,Collection028795006071002020021222242191530370...,Collection028795006071002020021222760971530394...,201880373.0
4,FormntcV3.frm02879500607060202008290913621500224,7/6/2020,4:44,O900454,106-23,NaN,NaN,[REDACTED] LISTED SUBJECT IS UNDER 17 OUTSIDE ...,Collection028795006070602020075702601541500153...,Collection028795006070602020075547896851500138...,201880373.0


In [393]:
q3nontraffic.shape[0]

315

In [394]:
q3warning = pd.read_csv('https://data.milwaukee.gov/dataset/930c7e1a-793f-451c-b1dc-ee55946a0545/resource/924f437b-75f7-442c-b17a-ef8b0eba7369/download/tracs-warning-joined.csv')
q3warning.head()

,PrdKey,documentPoliceNumber,summaryDateOccurred,summaryTimeOccurred,IndividualColKey,LocationColKey,agencyAgencySpace
0,FormwarningV3.0109380060706020200549514969445346,201881936.0,7/6/2020,17:49,Collection01093800607060202005502022836445405#...,Collection01093800607060202005531045337445587#...,NaN
1,FormwarningV3.01093800607070202003484365910445983,201891553.0,7/7/2020,15:48,Collection0109380060707020200349168179244545#U...,Collection01093800607070202003521285121445228#...,NaN
2,FormwarningV3.01093800607070202004110763480445522,201891624.0,7/7/2020,16:11,Collection01093800607070202004113947840445583#...,Collection01093800607070202004163017209445872#...,NaN
3,FormwarningV3.0109380060707020200451449353445649,201891728.0,7/7/2020,16:51,Collection01093800607070202004521777572445711#...,Collection01093800607070202004562761151445972#...,NaN
4,FormwarningV3.01093800607070202005020346296445354,201891759.0,7/7/2020,17:02,Collection01093800607070202005023522990445415#...,Collection01093800607070202005053087678445601#...,NaN


In [395]:
q3warning.shape[0]

3561

In [396]:
q3electronic = pd.read_csv('https://data.milwaukee.gov/dataset/930c7e1a-793f-451c-b1dc-ee55946a0545/resource/71aa9c2f-8b9c-4699-a92a-cf96b64f78cb/download/tracs-elci-joined.csv')
q3electronic.head()

,PrdKey,violationDateOccurred,violationTimeOccurred,violationLocalOrdinanceDescription,violationLocalOrdinanceNumber,violationStatuteDescription,violationStatuteNumber,agencyAgencySpace,LocationColKey,DefendantColKey,documentPoliceNumber
0,FormelciV3.frm00934500608100202011014577420230...,7/12/2020,5:34,NaN,NaN,T105610,346.63(1)(b),NaN,Collection00934500608100202011055663092309269#...,Collection009345006081002020110556531072309310...,201940470
1,FormelciV3.frm00934500609220202011422535607230...,8/17/2020,3:19,NaN,NaN,T106735,346.63(1)(b),NaN,Collection009345006092202020114527744992309721...,Collection009345006092202020114528819612309762...,202300245
2,FormelciV3.frm00934500609220202012404448589230989,8/19/2020,22:48,NaN,NaN,T105633,346.63(2)(A)2,I WAS DISPATCHED TO A VEHICLE CRASH. [REDACTED],Collection00934500609220202012441885472309190#...,Collection009345006092202020124418512872309231...,202322404
3,FormelciV3.frm00934500609220202012565423307230...,8/22/2020,21:21,NaN,NaN,T105610,346.63(1)(b),NaN,Collection00934500609220202001013066142309481#...,Collection009345006092202020010130772912309522...,202352277
4,FormelciV3.frm00934500609290202004204318022230...,9/5/2020,19:17,NaN,NaN,T105610,346.63(1)(b),NaN,Collection02264300609050202008482668528341420#...,Collection02264300609050202008482725290341461#...,202491864


In [397]:
q3electronic.shape[0]

8573

In [398]:
q3location = pd.read_csv('https://data.milwaukee.gov/dataset/930c7e1a-793f-451c-b1dc-ee55946a0545/resource/4d53b5f0-673a-4339-bb8b-abaae9bdcdac/download/tracs-location.csv')
q3location.head()

,CollKey,latitude,longitude
0,Collection00737100607030202001310046875311167#...,43.062000,-87.936336
1,Collection00737100607030202001372990050311237#...,43.061998,-87.936336
2,Collection00737100607030202001390351261311273#...,43.061998,-87.936336
3,Collection0073710060703020200140188932311372#U...,43.061998,-87.936336
4,Collection00737100607040202001413121099311248#...,43.026032,-87.992470


In [399]:
q3location.shape[0]

19944

## Combine 3 quarters' data into single tables

In [400]:
force = q1force.append([q2force, q3force])
force.shape[0]

174

In [401]:
summary=q1summary.append([q2summary, q3summary])
summary.shape[0]

29378

In [402]:
nontraffic=q1nontraffic.append([q2nontraffic, q3nontraffic])
nontraffic.shape[0]

2857

In [403]:
warning=q1warning.append([q2warning, q3warning])
warning.shape[0]

15561

In [404]:
electronic=q1electronic.append([q2electronic, q3electronic])
electronic.shape[0]

29849

In [405]:
#Combine summary, nontraffic, warning, and electronic into a single table, since their information is complimentary
details=summary.append([nontraffic, warning, electronic])
details.shape[0]

77645

In [406]:
location=q1location.append([q2location, q3location])
location.shape[0]

77225

## Format tables to facilitate inner joins with "keys" aka common columns

In [407]:
location.rename(columns={'CollKey': 'LocationColKey'}, inplace=True)
location.head()

,LocationColKey,latitude,longitude
0,Collection0051660060123020200844046856305581#U...,43.052939,-87.893371
1,Collection0051660060123020200858589603055352#U...,43.052648,-87.892626
2,Collection0051660060221020200325475758805557#U...,43.052940,-87.893257
3,Collection007369006021402020042548469051142293...,42.988300,-87.905472
4,Collection00737100601050202011145011392311397#...,43.026032,-87.992432


In [408]:
force.rename(columns={'CAD_Call_Number': 'documentPoliceNumber'}, inplace=True)
force.head()

,UOF_Report_Number,incident_date,documentPoliceNumber,Officer_Activity,Type_of_Force,IncidentPersonID,Justification_for_UOF
0,PF-2020-0010,1/4/2020,200040248.0,Effecting ArrestTraffic Stop,Bodily Force,Person 1,NaN
1,PF-2020-0018,1/13/2020,200132357.0,Effecting ArrestField Interview,Bodily Force,Person 1,NaN
2,PF-2020-0018,1/13/2020,200132357.0,Effecting ArrestField Interview,Bodily Force,Person 1,NaN
3,PF-2020-0018,1/13/2020,200132357.0,Effecting ArrestField Interview,Bodily Force,Person 1,NaN
4,PF-2020-0018,1/13/2020,200132357.0,Effecting ArrestField Interview,Bodily Force,Person 1,NaN


In [409]:
# remove decimal from documentPoliceNumber
force['documentPoliceNumber'] = force['documentPoliceNumber'].fillna(0).astype(np.int64)
force.head()

,UOF_Report_Number,incident_date,documentPoliceNumber,Officer_Activity,Type_of_Force,IncidentPersonID,Justification_for_UOF
0,PF-2020-0010,1/4/2020,200040248,Effecting ArrestTraffic Stop,Bodily Force,Person 1,NaN
1,PF-2020-0018,1/13/2020,200132357,Effecting ArrestField Interview,Bodily Force,Person 1,NaN
2,PF-2020-0018,1/13/2020,200132357,Effecting ArrestField Interview,Bodily Force,Person 1,NaN
3,PF-2020-0018,1/13/2020,200132357,Effecting ArrestField Interview,Bodily Force,Person 1,NaN
4,PF-2020-0018,1/13/2020,200132357,Effecting ArrestField Interview,Bodily Force,Person 1,NaN


In [410]:
summary.dtypes

PrdKey                      object
summaryDateOccurred         object
summaryTimeOccurred         object
summaryReason               object
summaryReasonDetail         object
summaryOtherReason          object
summaryOtherReasonDetail    object
summaryOutcome              object
agencyAgencySpace           object
documentPoliceNumber        object
LocationColKey              object
dtype: object

In [411]:
force['documentPoliceNumber'] = force['documentPoliceNumber'].astype(str)
force.head()

,UOF_Report_Number,incident_date,documentPoliceNumber,Officer_Activity,Type_of_Force,IncidentPersonID,Justification_for_UOF
0,PF-2020-0010,1/4/2020,200040248,Effecting ArrestTraffic Stop,Bodily Force,Person 1,NaN
1,PF-2020-0018,1/13/2020,200132357,Effecting ArrestField Interview,Bodily Force,Person 1,NaN
2,PF-2020-0018,1/13/2020,200132357,Effecting ArrestField Interview,Bodily Force,Person 1,NaN
3,PF-2020-0018,1/13/2020,200132357,Effecting ArrestField Interview,Bodily Force,Person 1,NaN
4,PF-2020-0018,1/13/2020,200132357,Effecting ArrestField Interview,Bodily Force,Person 1,NaN


In [412]:
# Drop duplicates so that one record per documentPoliceNumber remains
uniqueforce = force.drop_duplicates(subset='documentPoliceNumber', keep="first")
uniqueforce.head()

,UOF_Report_Number,incident_date,documentPoliceNumber,Officer_Activity,Type_of_Force,IncidentPersonID,Justification_for_UOF
0,PF-2020-0010,1/4/2020,200040248,Effecting ArrestTraffic Stop,Bodily Force,Person 1,NaN
1,PF-2020-0018,1/13/2020,200132357,Effecting ArrestField Interview,Bodily Force,Person 1,NaN
5,PF-2020-0021,1/19/2020,200192340,Dispatched AssignmentEffecting ArrestField Int...,Bodily Force,Person 1,NaN
6,PF-2020-0027,1/21/2020,200212621,Traffic Stop,Bodily Force,Person 1,NaN
8,PF-2020-0030,1/21/2020,200210761,Effecting ArrestField Interview,Bodily Force,Person 1,NaN


In [413]:
uniqueforce.shape[0]

104

## Combine the uniqueforce, details, and location tables into single table

In [414]:
locationdetails = pd.merge(details,location, on="LocationColKey")
locationdetails.head()

,PrdKey,summaryDateOccurred,summaryTimeOccurred,summaryReason,summaryReasonDetail,summaryOtherReason,summaryOtherReasonDetail,summaryOutcome,agencyAgencySpace,documentPoliceNumber,LocationColKey,violationDateOccurred,violationTimeOccurred,violationLocalOrdinanceDescription,violationLocalOrdinanceNumber,violationStatuteDescription,violationStatuteNumber,DefendantColKey,IndividualColKey,latitude,longitude
0,Formcontactsum00737100601280202008461462274311114,1/28/2020,8:43,TS01,VR01,NaN,NaN,"02,03","DEF DRINVING LISTED AUTO, S/B ON N. 27TH ST @ ...",200280642,Collection00737100601280202008461534347311115#...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,43.045115,-87.947759
1,Formcontactsum00737100602060202001343369944311473,2/6/2020,13:06,TS01,"OR01,SV01",NaN,NaN,4,"I OBSERVED DEF SPEEDING 51MPH ,30 SPEED ZONE A...",200371284,Collection00737100602060202001253863515311237#...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,43.051519,-87.963678
2,Formcontactsum00737100602100202001512366734311905,2/10/2020,13:42,TS01,SV01,NaN,NaN,4,"DEF. DRIVING LISTED AUTO,STOPPED FOR SPEEDING....",200411328,Collection00737100602100202001512443096311906#...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,43.134045,-87.995137
3,Formcontactsum00818800601300202008440868943278730,1/30/2020,8:37,TS01,SV01,NaN,NaN,3,LISTED SUBJECT WAS OBSERVED TRAVELING ABOVE TH...,200300628,Collection00818800601300202008400444419278489#...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,43.090058,-87.992444
4,Formcontactsum0083920060206020201104446856377978,2/6/2020,7:20,FI01,99,NaN,SUSPECT INVOLVED IN A DISORDERLY ACT,4,SUBJECT WENT INTO HER CHILD SCHOOL LOOKING FOR...,200370549,Collection0083920060206020201108162775477982#U...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,43.077421,-87.941078


In [415]:
locationdetails.shape[0]

79816

In [416]:
locationdetails.dtypes

PrdKey                                 object
summaryDateOccurred                    object
summaryTimeOccurred                    object
summaryReason                          object
summaryReasonDetail                    object
summaryOtherReason                     object
summaryOtherReasonDetail               object
summaryOutcome                         object
agencyAgencySpace                      object
documentPoliceNumber                   object
LocationColKey                         object
violationDateOccurred                  object
violationTimeOccurred                  object
violationLocalOrdinanceDescription     object
violationLocalOrdinanceNumber          object
violationStatuteDescription            object
violationStatuteNumber                 object
DefendantColKey                        object
IndividualColKey                       object
latitude                              float64
longitude                             float64
dtype: object

In [417]:
uniqueforce.dtypes

UOF_Report_Number        object
incident_date            object
documentPoliceNumber     object
Officer_Activity         object
Type_of_Force            object
IncidentPersonID         object
Justification_for_UOF    object
dtype: object

In [418]:
uniqueforcedata=pd.merge(uniqueforce, locationdetails, on="documentPoliceNumber", how="inner")
uniqueforcedata.head()

,UOF_Report_Number,incident_date,documentPoliceNumber,Officer_Activity,Type_of_Force,IncidentPersonID,Justification_for_UOF,PrdKey,summaryDateOccurred,summaryTimeOccurred,summaryReason,summaryReasonDetail,summaryOtherReason,summaryOtherReasonDetail,summaryOutcome,agencyAgencySpace,LocationColKey,violationDateOccurred,violationTimeOccurred,violationLocalOrdinanceDescription,violationLocalOrdinanceNumber,violationStatuteDescription,violationStatuteNumber,DefendantColKey,IndividualColKey,latitude,longitude
0,PF-2020-0010,1/4/2020,200040248,Effecting ArrestTraffic Stop,Bodily Force,Person 1,NaN,Formcontactsum0246200060104020200544002529889235,1/4/2020,2:13,TS01,VE01,NaN,NaN,5,ATTEMPTED A TRAFFIC STOP FOR DEFECTIVE REG LIG...,Collection02462000601040202005404068173889126#...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,43.069724,-87.959958
1,PF-2020-0010,1/4/2020,200040248,Effecting ArrestTraffic Stop,Bodily Force,Person 1,NaN,FormelciV3.frm02462000601040202005403990710889101,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Collection02462000601040202005404068173889126#...,1/4/2020,2:13,O900898,101-1-2,T105825,347.13(3),Collection0246200060104020200529516162988978#U...,NaN,43.069724,-87.959958
2,PF-2020-0010,1/4/2020,200040248,Effecting ArrestTraffic Stop,Bodily Force,Person 1,NaN,FormelciV3.frm0246200060104020200436113743588947,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Collection02462000601040202005404068141889128#...,1/4/2020,2:13,O900023,CIRCUIT,T104909,346.04(3),Collection02462000601040202005404032226889176#...,NaN,43.069724,-87.959958
3,PF-2020-0050,2/1/2020,200321753,Traffic Stop,Bodily Force,Person 1,NaN,Formcontactsum02849500602120202010031138509116...,2/1/2020,17:02,TS01,VR01,NaN,NaN,5,LISTED DEFENDANT WAS OPERATING THE LISTED VEHI...,Collection028495006021202020100336203291164840...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,43.024169,-87.947999
4,PF-2020-0050,2/1/2020,200321753,Traffic Stop,Bodily Force,Person 1,NaN,FormntcV3.frm0284950060201020200938186439685772,NaN,NaN,NaN,NaN,NaN,NaN,NaN,LISTED DEFENDANT WAS FOUND TO BE IN POSSESSION...,Collection02849500602010202009444386048857139#...,2/1/2020,17:02,O900385,106-38-2,NaN,NaN,Collection02849500602010202009385584067857100#...,NaN,43.024170,-87.948052


In [419]:
uniqueforcedata.shape[0]

164

In [420]:
uniqueforcedata.dropna(subset=['latitude', 'longitude'], inplace=True)
uniqueforcedata.shape[0]

146

## Gather & format data - then make the heatmap

In [421]:
# Milwaukee latitude and longitude values
latitude = 43.05
longitude = -87.92

In [422]:

# Create map
UseOfForce_map = folium.Map(
    location=[43.05, -87.92],
    tiles='cartodbpositron',
    zoom_start=11)

# display the map
UseOfForce_map

In [423]:
# Link lat/long values from dataframe to map

locs=uniqueforcedata[['latitude','longitude']]
loc_list = locs.values.tolist()
len(loc_list)
loc_list[7]


[43.02417055, -87.9479985]

In [424]:
#len(loc_list)

for point in range(0, 146):
    folium.Marker(loc_list[point]).add_to(UseOfForce_map)
    
# display the map
UseOfForce_map

In [425]:
uniqueforcedata['count'] = 1
ufheat_map = folium.Map(
    location=[43.05, -87.92],
    tiles='cartodbpositron',
    zoom_start=11)
HeatMap(data=uniqueforcedata[['latitude', 'longitude', 'count']].groupby(['latitude', 'longitude']).sum().reset_index().values.tolist(), radius=10, max_zoom=13).add_to(ufheat_map)

In [426]:
ufheat_map